# CNN Traffic Sign

CNN, Convolutional Neuronal Network using pythorch to identify European traffic signs for competition

In [2]:
# Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torchvision.datasets as datasets
import torchvision.transforms as T
import matplotlib.pyplot as plt

## USING GPU

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

cuda


In [4]:
# Constants
img_path = 'IMG_SETs/' # Path to the image data

NUM_TRAIN = 2500 # Number of training samples
NUM_VAL = 500 # Number of validation samples
NUM_TEST = 500 # Number of test samples
BATCH_SIZE = 100 # Batch size

# imgaes are 3x640x640

# Normalize the image data
transform = T.Compose([
    T.ToTensor(),
    T.Normalize([0.491, 0.482, 0.447], [0.247, 0.243, 0.261])
])

# Load your dataset
dataset = datasets.ImageFolder(img_path, transform=transform)

# Split the dataset into train, validation, and test sets
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [NUM_TRAIN, NUM_VAL, NUM_TEST])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

FileNotFoundError: Found no valid file for the classes Stop, Turn_Left. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

## Acurracy

In [5]:
def accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            scores = model(x)
            _, predictions = scores.max(dim=1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
    return float(num_correct)/num_samples

## Traingin loop

In [ ]:
def train(model,optimiser, epochs = 100):
    model = model.to(device=device)
    for epochs in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()
            x = x.to(device=device, dtype=torch.float32)
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            loss = F.cross_entropy(scores, y)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            if t % 100 == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                print('Validation accuracy = %.2f' % accuracy(val_loader, model))
                print()
            
    return model

## CNN Class


In [8]:
conv_k_3 = lambda channel1, channel2: nn.Conv2d(channel1, channel2, kernel_size=3, padding=1)

class CNN_class(nn.Module):
    def __init__(self, in_channel, channel1, channel2):
        super().__init__()
        self.conv1 = conv_k_3(in_channel, channel1)
        self.bn1 = nn.BatchNorm2d(channel1)

        self.conv2 = conv_k_3(channel1, channel2)
        self.bn2 = nn.BatchNorm2d(channel2)
        
        self.max_pool = nn.MaxPool2d(2,2)

    def forward(self, x):
        x = F.relu(self.bn2(self.conv2(F.relu(self.bn1(self.conv1(x))))))
        return self.max_pool(x)

In [ ]:
# Define the model
channel1 = 16
channel2 = 32
channel3 = 64
channel4 = 128

epochs = 100
lr = 1e-3

model = nn.Sequential(CNN_class(3, channel1, channel2),
                      CNN_class(channel2, channel3, channel4),
                      nn.Flatten(),
                      nn.Linear(128*40*40, 3))

optimiser = torch.optim.Adam(model.parameters(), lr=lr)

## Train

In [ ]:
train(model, optimiser, epochs)

In [ ]:
accuracy(test_loader, model)

## Exporting Model